# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. <br><br>
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных

In [5]:
# импортирование библиотеки 'pandas' с последующим обращением как 'pd'
import pandas as pd 

# сохранение датасета в переменную data
data = pd.read_csv('dataset_bank_clients.csv', index_col=0) 

# получение информации о датафрейме
display(data) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Шаг 2.1 Заполнение пропусков

In [2]:
data.info() # общая информация о датафрейме

count_of_none = data['days_employed'].isna().sum() # количество пропущенных значений в столбцах 'days_employed', 'total_income'
count_of_values = data['children'].value_counts().sum() # количество значений 

display(count_of_none / count_of_values * 100) # доля пропущенных значений в столбцах 'days_employed', 'total_income' 
                                               # составляет 10% от общего числа значений

median_days_employed = data['days_employed'].median() # медианное значение столбца 'days_employed'
median_total_income = data['total_income'].median() # медианное значение столбца 'total_income'

data['days_employed'] = data['days_employed'].fillna(value=median_days_employed) # замена пропусков на медианное 
                                                                                 # значение в столбце 'days_employed'
data['total_income'] = data['total_income'].fillna(value=median_total_income) # замена пропусков на медианное 
                                                                              # значение в столбце 'total_income'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


10.099883855981417

### Шаг 2.2 Проверка данных на аномалии и исправления.

После просмотра данных выяснилось, что только у пенсионеров и безработных трудовой стаж положителен
по-прежнему остается непонятным почему присутствуют отрицательные значения, также неизвестно почему 
положительный трудовой стаж у пенсионеров и безработных нереалистично большой, если переводить его в года. <br>

Теория: стаж для работающих людей был подсчитан следующим образом: 
(дата начала работы переведенная в дни) - (сегодняшняя дата переведенная в дни). <br>
Как подсчитывался стаж у пенсионеров и безработных - неизвестно.

In [3]:
display(data.value_counts('children'))

children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
dtype: int64

In [4]:
display(data[data['days_employed'] < 0].value_counts('days_employed'))

days_employed
-1203.369529     2175
-18388.949901       1
-1012.751273        1
-1015.061031        1
-1014.723386        1
                 ... 
-2540.825433        1
-2540.721375        1
-2540.655487        1
-2540.649251        1
-24.141633          1
Length: 15906, dtype: int64

Аномалии в данных: 20 детей, отрицательный стаж.

In [5]:
def fix_days_employed(row):  # функция, исправляющая стаж у работающих и не работающих
    days_employed = row['days_employed']
    income_type = row['income_type']
    
    if days_employed < 0:
        row['days_employed'] = row['days_employed'] * (-1)
    
    elif (income_type == 'пенсионер') or (income_type == 'безработный'):
        row['days_employed'] = row['days_employed'] / 20
        days_employed = row['days_employed']
        
        if ((row['dob_years'] - (row['days_employed'] / 365)) < 18) and (row['dob_years'] != 0):
            row['days_employed'] = (row['dob_years'] - 18) * 365
            
    return row
    
data = data.apply(fix_days_employed, axis=1) # применение функции ко всему датафрейму по строкам 

display(len(data[data['dob_years'] == 0]) / len(data) * 100) # количество клиентов с нулевым возрастом 
                                                             # составляет 0.5% от всего датафрейма
data = data[data['dob_years'] != 0] # удаление информации о клиентах с пропущенным возрастом

display(len(data[data['children'] == -1]) / len(data) * 100) # количество клиентов с отрицательным
                                                             # количеством детей составляет 0,2%
data = data[data['children'] != -1] # удаление информации о клиентах с отрицательным количеством детей 


0.4692218350754936

0.21938013442867813

### Шаг 2.3. Изменение типов данных.

In [6]:
# изменение типа данных в столбце 'total_income' с вещественного на целочисленный 
data['total_income'] = data['total_income'].astype('int') 

### Шаг 2.4. Удаление дубликатов.
Скорее всего дубликаты появились из-за некорректного заполнения датафрейма (без обращения к ТЗ).

In [6]:
# все значения столбца 'education'
display(data['education'].value_counts()) 

# приведение значений к нижнему регистру
data['education'] = data['education'].str.lower() 

# подсчет количества дубликатов
display(data.duplicated().sum())  

 # удаление дубликатов с формированием новых индексов
data = data.drop_duplicates().reset_index(drop=True)

# повторная проверка столбца на наличие дубликатов
display(data['education'].value_counts()) 

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

71

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [7]:
data_education = pd.DataFrame({'education':data['education'], 'education_id':data['education_id']}).drop_duplicates().reset_index(drop=True)
display(data_education)

data_family = pd.DataFrame({'family_status':data['family_status'], 'family_status_id':data['family_status_id']}).drop_duplicates().reset_index(drop=True)
display(data_family)

data = data.drop(columns=['education', 'family_status'], axis=1) # удаление столбцов 'education', 'family_status'

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


### Шаг 2.6. Категоризация дохода.

In [8]:
def set_total_income_category(row): # функция для определения категории дохода
    total_income = row['total_income']
    
    if (total_income > 0) and (total_income <= 30000):
        return 'E'
    elif (total_income > 30000) and (total_income <= 50000):
        return 'D'
    elif (total_income > 50000) and (total_income <= 200000):
        return 'C'
    elif (total_income > 200000) and (total_income <= 1000000):
        return 'B'
    elif total_income > 1000000:
        return 'A'

# применение функции ко всему датафрейму по строкам    
data['total_income_category'] = data.apply(set_total_income_category, axis=1) 
display(data)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875.639453,покупка жилья,B
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080.014102,приобретение автомобиля,C
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885.952297,покупка жилья,C
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628.550329,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616.077870,сыграть свадьбу,C
...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529.316663,43,1,1,F,компаньон,0,224791.862382,операции с жильем,B
21450,0,343937.404131,67,1,0,F,пенсионер,0,155999.806512,сделка с автомобилем,C
21451,1,-2113.346888,38,1,1,M,сотрудник,1,89672.561153,недвижимость,C
21452,3,-3112.481705,38,1,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,B


Пропусков в столбце 'total_income_category' не наблюдается, значит все данные категоризированы.

In [10]:
display(data.value_counts('total_income_category'))

total_income_category
C    15905
B     5007
D      347
A       25
E       22
dtype: int64

### Шаг 2.7. Категоризация целей кредита.

In [9]:
def set_purpose_category(row): # функция для определения категории операции
    purpose = row['purpose']
    
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    elif ('недвижимо' in purpose) or ('жил' in purpose):
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образован' in purpose:
        return 'получение образования'

# применение функции ко всему датафрейму по строкам
data['purpose_category'] = data.apply(set_purpose_category, axis=1) 

### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

##### Вывод 1:

<b>
У людей с детьми и с прошлыми задолжностями, по увеличению количества детей никакого восходящего тренда не наблюдается. Заметно, что в среднем людей с детьми и прошлыми задолжностями больше 1.5-2%
</b>

Первый столбец 'count' показывает количество заемщиков с соответствующим количеством детей, <br>
второй столбец 'count_with_debt' показывает количество заемщиков с соответствующим количеством детей и с наличием долга, <br>
третий столбец 'percent' показывает процентное соотношение этих двух величин.

In [12]:
data_res1 = data.pivot_table(index='children', values='debt', aggfunc=['count', 'sum'])

 # переименование двухэтажных столбцов 
data_res1.columns = ['count', 'count_with_debt']

# вычисление процентного соотношения 
data_res1['percent'] = data_res1['count_with_debt'] / data_res1['count'] * 100 

display(data_res1)

,count,count_with_debt,percent
children,,,
0,14022,1058,7.545286
1,4792,441,9.202838
2,2039,194,9.514468
3,328,27,8.231707
4,41,4,9.756098
5,9,0,0.000000
20,75,8,10.666667


##### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

##### Вывод 2:

<b>
Определенной зависимости между семейным положением и наличием прошлых долгов не выявлено
</b>

In [13]:
data_res1 = data.pivot_table(index='family_status_id', values='debt', aggfunc=['count', 'sum'])
data_res1.columns = ['count', 'count_with_debt'] # переименование двухэтажных столбцов 
data_res1['percent'] = data_res1['count_with_debt'] / data_res1['count'] * 100 # вычисление процентного соотношения 

display(data_res1)

,count,count_with_debt,percent
family_status_id,,,
0,12261,926,7.552402
1,4125,386,9.357576
2,950,62,6.526316
3,1181,85,7.197290
4,2789,273,9.788455


##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

##### Вывод 3:

<b>
Люди с доходом менее 30 тысяч имеют наибольшее значение в соотношении 'люди с прошлой задолжностью ко всем людям с таким доходом', что говорит о наименьшей способности к своевременному возврату долга. Самой отвественной группой в этом плане стали люди с доходом от 30 до 50 тысяч (наименьшее значение соотношения). Разброс значений (3%) и отсутствие восходящего или нисходящего тренда показывают, что ярковыраженной зависимости между уровнем дохода и возвратом кредита вовремя нет. 
</b>

In [14]:
data_res1 = data.pivot_table(index='total_income_category', values='debt', aggfunc=['count', 'sum'])
data_res1.columns = ['count', 'count_with_debt'] # переименование двухэтажных столбцов 
data_res1['percent'] = data_res1['count_with_debt'] / data_res1['count'] * 100 # вычисление процентного соотношения 

display(data_res1)

,count,count_with_debt,percent
total_income_category,,,
A,25,2,8.000000
B,5007,355,7.090074
C,15905,1352,8.500472
D,347,21,6.051873
E,22,2,9.090909


##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

##### Вывод 4:

<b>
Клиенты, которые берут кредит с целью получения образования, как и клиенты, которые берут кредит с целью покупки автомобиля в среднем возвращают долги на 1,5-2% реже, чем клиенты, которые берут кредит на проведение свадьбы или с целью покупки недвижимости.
</b>

In [10]:
data_res1 = data.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'sum'])

# переименование двухэтажных столбцов 
data_res1.columns = ['count', 'count_with_debt'] 

# вычисление процентного соотношения 
data_res1['percent'] = data_res1['count_with_debt'] / data_res1['count'] * 100 

display(data_res1)

,count,count_with_debt,percent
purpose_category,,,
операции с автомобилем,4306,403,9.359034
операции с недвижимостью,10811,782,7.233373
получение образования,4013,370,9.220035
проведение свадьбы,2324,186,8.003442


## Общий вывод:

Влияние количества детей на возврат кредита в срок: не выявлено, но заемщики с детьми на 1,5-2% реже отдают кредиты в срок, как следствие,<b> следует понизить кредитный рейтинг людей с детьми</b> <br>		
    
Овдовевшие люди имеют наибольший процент возврата кредита в срок, <b> им следует повысить кредитный рейтинг </b> <br>
    
Клиенты с доходом менее 30 тысяч меньше всего способны на возврат кредита в срок, люди с доходом от 30 до 50 тысяч имеют наибольший процент возврата кредита в срок,<b> группе с доходом менее 30 тысяч - понизить кредитный рейтнг, группе с доходом от 30 до 50 тысяч - повысить  </b> <br>	
    
Клиентам с целями получения кредита на 'операции с автомобилем', 'получение образования' выявлен наименьший показатель возврата кредита в срок, <b> им следует понизить кредитный рейтинг </b> <br>